In [10]:
import numpy as np
def tensor_to_blob(data):
    psd = data['data']['psd']  # Extract PSD tensor
    psd = psd.numpy().astype(np.float32)  # Ensure 32-bit float format
    return psd.tobytes()  # Convert NumPy array to binary (BLOB)



In [11]:
import torch 
import numpy as np
def custom_collate_fn(batch):
    """
    Custom collate function to combine data from multiple files into a single batch.
    Each field is handled appropriately.
    """
    # Initialize lists to hold batched data
    file_paths = []
    dates = []
    corrupted_flags = []
    time_series = []
    fs_list = []
    channel_names_list = []
    channel_units_list = []
    levels = []
    directions = []
    sensors = []

    # Iterate over each data dictionary in the batch
    for data in batch:
        file_data = data['file_data']
        time_series.extend((file_data['data']))  # Assuming data is converted to tensor for processing
        number_of_samples = len(file_data['data'])
        file_paths.append(data['file_path'])
        dates.extend([data['date']]*number_of_samples)
        corrupted_flags.extend([data['file_corrupted']]*number_of_samples)
        
        # Access the file_data dictionary for relevant fields
        fs_list.append(file_data['fs'])
        channel_names_list.extend(file_data['channel_names'])
        channel_units_list.extend(file_data['channel_units'])
        levels.extend(file_data['level'])
        
        # Handle directions and sensors, ensuring None values are preserved
        directions.extend(file_data['direction'])
        sensors.extend(file_data['sensor'])

    # Convert lists to tensors where appropriate (e.g., psd_data)
    time_series = torch.from_numpy(np.stack(time_series,dtype=np.float32))
    data= {'time_series': time_series,
           'channel_names': channel_names_list, 'channel_units': channel_units_list, 
           'levels': levels, 'directions': directions, 'sensors': sensors,
           'corrupted_flags': corrupted_flags, 'dates': dates}

    # Combine the data into a single dictionary for the batch
    batch_data = {
        'file_paths': file_paths,
        'fs': fs_list,
        'data': data 
    }

    return batch_data

In [2]:
from config import settings
from torchdata.datapipes import iter as it 
from matplotlib import pyplot as plt
import torch
from hannover_pylon.data_t import utils as data_utils
from hannover_pylon.data_t import preprocessing as pp 
FS = 1651
NFFT = 16392
dp = it.FileLister(root= settings.path.raw,recursive=True,masks='*.mat')
dp = dp.map(data_utils.process_file_path)
dp_acc, dp_meta_met, dp_meta_struct, dp_corrupted = dp.demux(num_instances=4, \
        classifier_fn= data_utils.demux_data,buffer_size=70_000)
dp_corrupted = dp_corrupted.map(data_utils.readfile)
tracker = data_utils.TrackCorruptedFiles()
dp_corrupted = dp_corrupted.map(tracker)
list(dp_corrupted)
tracker.setup()
dp_acc = dp_acc.map(tracker)
dp_acc = dp_acc.map(data_utils.readfile)
dp_acc = dp_acc.map(data_utils.extract_channel_info)
dp_acc = dp_acc.batch(30)
dp_acc = dp_acc.collate(custom_collate_fn)
dp_meta_met = dp_meta_met.map(data_utils.readfile)
dp_meta_met = dp_meta_met.map(data_utils.extend_date)
welch = pp.Welch(n_fft=NFFT, fs=FS)
dp_acc = dp_acc.map(welch)
ds = next(iter(dp_acc))


/home/owilab/Documents/YacineB/Code/hannover-pylon/.venv/lib/python3.10/site-packages/torchdata/datapipes/__init__.py:18: UserWarning: 
################################################################################
WARNING!
The 'datapipes', 'dataloader2' modules are deprecated and will be removed in a
future torchdata release! Please see https://github.com/pytorch/data/issues/1196
to learn more and leave feedback.
################################################################################

  deprecation_warning()


In [ ]:
import sqlite3
from pathlib import Path
db_path = Path(settings.path.processed, 'data.db')
conn = sqlite3.connect(db_path)
c = conn.cursor()
c.execute('''CREATE TABLE IF NOT EXISTS data
             (id INTEGER PRIMARY KEY,
             psd BLOB,
             date TEXT,
             level TEXT,
            direction TEXT,
            corrupted BOOLEAN,
            sensor_type TEXT)''')
conn.commit()
insert_query = '''INSERT INTO data (psd, date, level, direction, corrupted, sensor_type) VALUES (?, ?, ?, ?, ?, ?)'''
conn.execute('BEGIN TRANSACTION')
for batch in dp_acc:

    date_batch = batch['data']['dates']
    psd_batch = batch['data']['log_psd']
    level_batch = batch['data']['levels']
    direction_batch = batch['data']['directions']
    corrupted_batch = batch['data']['corrupted_flags']
    sensor_batch = batch['data']['sensors']
    psd_serialized = [psd.cpu().numpy().astype('float32').tobytes() for psd in psd_batch]
    c.executemany(insert_query, list(zip(psd_serialized, date_batch, sensor_batch,level_batch, direction_batch, corrupted_batch)))
    conn.commit()
conn.execute('END TRANSACTION')





'//home/owilab/Documents/YacineB/Code/hannover-pylon/data/processed'

In [ ]:
DROP TABLE IF EXISTS "reshaped";

CREATE TABLE "reshaped" AS
SELECT
  "data"."date" AS "date",
  MAX(CASE WHEN sensor = 'accel' AND level = 1 AND direction = 'x' THEN psd END) AS "psd_level1_x",
  MAX(CASE WHEN sensor = 'accel' AND level = 1 AND direction = 'y' THEN psd END) AS "psd_level1_y",
  MAX(CASE WHEN sensor = 'accel' AND level = 2 AND direction = 'x' THEN psd END) AS "psd_level2_x",
  MAX(CASE WHEN sensor = 'accel' AND level = 2 AND direction = 'y' THEN psd END) AS "psd_level2_y",
  MAX(CASE WHEN sensor = 'accel' AND level = 3 AND direction = 'x' THEN psd END) AS "psd_level3_x",
  MAX(CASE WHEN sensor = 'accel' AND level = 3 AND direction = 'y' THEN psd END) AS "psd_level3_y",
  MAX(CASE WHEN sensor = 'accel' AND level = 4 AND direction = 'x' THEN psd END) AS "psd_level4_x",
  MAX(CASE WHEN sensor = 'accel' AND level = 4 AND direction = 'y' THEN psd END) AS "psd_level4_y",
  MAX(CASE WHEN sensor = 'accel' AND level = 5 AND direction = 'x' THEN psd END) AS "psd_level5_x",
  MAX(CASE WHEN sensor = 'accel' AND level = 5 AND direction = 'y' THEN psd END) AS "psd_level5_y",
  MAX(CASE WHEN sensor = 'accel' AND level = 6 AND direction = 'x' THEN psd END) AS "psd_level6_x",
  MAX(CASE WHEN sensor = 'accel' AND level = 6 AND direction = 'y' THEN psd END) AS "psd_level6_y",
  MAX(CASE WHEN sensor = 'accel' AND level = 7 AND direction = 'x' THEN psd END) AS "psd_level7_x",
  MAX(CASE WHEN sensor = 'accel' AND level = 7 AND direction = 'y' THEN psd END) AS "psd_level7_y",
  MAX(CASE WHEN sensor = 'accel' AND level = 8 AND direction = 'x' THEN psd END) AS "psd_level8_x",
  MAX(CASE WHEN sensor = 'accel' AND level = 8 AND direction = 'y' THEN psd END) AS "psd_level8_y",
  MAX(CASE WHEN sensor = 'accel' AND level = 9 AND direction = 'x' THEN psd END) AS "psd_level9_x",
  MAX(CASE WHEN sensor = 'accel' AND level = 9 AND direction = 'y' THEN psd END) AS "psd_level9_y"
FROM "data"
GROUP BY "data"."date";
